# Deep Learning Final - RNN

##### -Sela Koshy

0. Add required packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


1.0 Load and process the dataset

In [2]:
data = pd.read_csv('Global_Temperatures.csv')

data['dt'] = pd.to_datetime(data['dt'])
data['year'] = data['dt'].dt.year
data['month'] = data['dt'].dt.month
data = data[['year', 'month', 'LandAverageTemperature']].copy()

1.1 Create training, validation and test sets

In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)
train_data, val_data = train_test_split(train_data, test_size=0.2, shuffle=False)

1.2 Separate the features, and make target variables

In [4]:
X_train = train_data[['year', 'month']].values
y_train = train_data['LandAverageTemperature'].values
X_val = val_data[['year', 'month']].values
y_val = val_data['LandAverageTemperature'].values
X_test = test_data[['year', 'month']].values
y_test = test_data['LandAverageTemperature'].values


X_train = X_train.reshape((-1, 1, 2))
X_val = X_val.reshape((-1, 1, 2))
X_test = X_test.reshape((-1, 1, 2))

2. Define model architecture

In [5]:
model = Sequential()
model.add(SimpleRNN(units=64, activation='tanh', input_shape=(1, 2)))
model.add(Dense(units=1))

3. Compile model, then train model

In [6]:
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32,
                    callbacks=[early_stopping, checkpoint])

Epoch 1/100
64/64 [==============================] - 3s 17ms/step - loss: 64.1927 - val_loss: 44.5706
Epoch 2/100
64/64 [==============================] - 0s 7ms/step - loss: 31.4966 - val_loss: 24.4098
Epoch 3/100
64/64 [==============================] - 0s 4ms/step - loss: 21.9676 - val_loss: 19.1822
Epoch 4/100
64/64 [==============================] - 0s 4ms/step - loss: 20.1593 - val_loss: 17.9616
Epoch 5/100
64/64 [==============================] - 0s 4ms/step - loss: 19.9934 - val_loss: 17.7358
Epoch 6/100
64/64 [==============================] - 0s 3ms/step - loss: 19.9625 - val_loss: 17.7378
Epoch 7/100
64/64 [==============================] - 0s 5ms/step - loss: 19.9533 - val_loss: 17.6787
Epoch 8/100
64/64 [==============================] - 0s 4ms/step - loss: 19.9619 - val_loss: 17.7384
Epoch 9/100
64/64 [==============================] - 0s 5ms/step - loss: 19.9572 - val_loss: 17.7067
Epoch 10/100
64/64 [==============================] - 0s 4ms/step - loss: 19.9598 - val_lo

4. Evaluate the model

In [7]:
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

20/20 [==============================] - 0s 2ms/step - loss: 18.1361
Test loss: 18.136072158813477


5. Save the model

In [8]:
model.save('global_temp_model.h5')